# Neural network Spam filtering using forward backward propagation

Used Ling spam data set from the links provided in the assignment statements

# Data extraction

In [47]:
#Required libraires for the project
import os
import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import re
train_text_list=[]
train_label_list=[]
test_text_list=[]
test_label_list=[]
#created train and test datasets from spam datset
train_df = pd.DataFrame(columns=['Text','Label'])
test_df = pd.DataFrame(columns=['Text','Label'])
count=0
spam_count=0
non_spam_count=0
for root, dirs, files in os.walk("Train/"):
        for  root, dirs, files in os.walk(root):
            for file in files:
                #if (count > 0):
                    f = open(root+"/"+file, "r",encoding='latin-1', errors='ignore')
                    text= f.read()
                    if (file[0:4]=="spms"):
                        lable= "spam"
                        spam_count=spam_count+1
                    else:
                        lable= "not_spam"
                        non_spam_count=non_spam_count+1
                    f.close()
                    train_text_list.append(text[8:])
                    train_label_list.append(lable)

train_df['Text']=train_text_list
train_df['Label']=train_label_list
#Creating test into different data frame
for root, dirs, files in os.walk("Test/"):
        for  root, dirs, files in os.walk(root):
            for file in files:
                #if (count > 0):
                    f = open(root+"/"+file, "r",encoding='latin-1', errors='ignore')
                    text= f.read()
                    if (file[0:4]=="spms"):
                        lable= "spam"
                        spam_count=spam_count+1
                    else:
                        lable= "not_spam"
                        non_spam_count=non_spam_count+1
                    f.close()
                    test_text_list.append(text[8:])
                    test_label_list.append(lable)
test_df['Text']=test_text_list
test_df['Label']=test_label_list

# Start of preprocessing

In [48]:
#Made text to lower case in train and test as apart of prep processing
train_df['Text'] = [entry.lower() for entry in train_df['Text']]
test_df['Text'] = [entry.lower() for entry in test_df['Text']]

In [49]:
#display tail (end 5 entries of train data)
train_df.tail()

,Text,Label
944,call : field report / endanger lg . lsa ' 96\...,not_spam
945,re : 6 . 139 word own opposite\n\ncurious cas...,not_spam
946,summer school program\n\ninternational summer...,not_spam
947,web development offer . . . open immediately\...,spam
948,summary sociolinguistic course syllabus\n\nde...,not_spam


In [50]:
#display tail (end 5 entries of test data)
test_df.tail()

,Text,Label
294,sla conference pari 1999\n\ncall for papers x...,not_spam
295,conference announcement\n\nsouthern illinoi u...,not_spam
296,semantic\n\nwe would like to bring to your at...,not_spam
297,released vol . 2\n\nreleased ! ! ! introducin...,spam
298,web development offer . . . open immediately\...,spam


In [51]:
#Added one more column to get number if of each row in data frame
ind_list=[]
ind_count=0
for i in train_df['Text']:
    ind_count=ind_count+1
    ind_list.append(ind_count)
train_df['Ind']= ind_list

In [52]:
#Added one more column to get number if of each row in data frame
ind_list1=[]
ind_count1=0
for i in test_df['Text']:
    ind_count1=ind_count1+1
    ind_list1.append(ind_count1)
test_df['Ind']= ind_list1

Data preprocessing and statistics

In [53]:
train_df.head()

,Text,Label,Ind
0,here\n\nuniversity degree programs increase p...,spam,1
1,bisfai deadline extension !\n\nbisfai deadlin...,not_spam,2
2,"incredible opportunity\n\ndear friend , recei...",spam,3
3,""" life without debt ""\n\npardon intrusion . o...",spam,4
4,"query :\n\ndear linguist , work double negati...",not_spam,5


In [54]:
print(train_df.shape)
print(train_df.Label.shape)

(949, 3)
(949,)


In [55]:
print(test_df.shape)
print(test_df.Label.shape)

(299, 3)
(299,)


based on labes got the count of spam and non spam

In [56]:
#Getting counts spam and not spam for preprocssing
spam_count=0
non_spam_count=0
for i in train_df['Label']:
    if (i== str('not_spam')):
        non_spam_count=non_spam_count+1
    if (i== str('spam')):
        spam_count= spam_count+1
        
print('non_spam_count', non_spam_count)
print('spam_count',spam_count)

non_spam_count 468
spam_count 481


In [57]:
#for just a check on data if the data frame is properly prepared
train_df['Label'].unique()

array(['spam', 'not_spam'], dtype=object)

In [58]:
train_df.describe(exclude=[np.number])#checking some stats of data

,Text,Label
count,949,949
unique,934,2
top,re :\n\n= 20 virtual girlfriend virtual boyfr...,spam
freq,4,481


In [59]:
#Making one more lab with 0 as Not spam and 1 as spam
train_df["label_tag"]= train_df['Label'].map( {'spam': 1, 'not_spam': 0} ).astype(int)
test_df["label_tag"]= train_df['Label'].map( {'spam': 1, 'not_spam': 0} ).astype(int)

Imported required modules for some staistics and to calculate the common words in the data set

In [60]:
#Imported required counters
from collections import Counter
import numpy as np
spam_counts = Counter()
not_spam_counts = Counter()
total_counts = Counter()
spam_not_ratios = Counter()

Applied word tokenize to convert the data into numbered features

In [61]:
#Word tokenize the raw text for train and test data as a part of preprocessing
from nltk import word_tokenize
train_df['Text'].dropna(inplace=True)
train_df['tokenized_text'] =train_df['Text'].apply(word_tokenize)
test_df['Text'].dropna(inplace=True)
test_df['tokenized_text'] =test_df['Text'].apply(word_tokenize)

Training data after tokenization to remove stop words and some special characters

In [62]:
train_df.head()

,Text,Label,Ind,label_tag,tokenized_text
0,here\n\nuniversity degree programs increase p...,spam,1,1,"[here, university, degree, programs, increase,..."
1,bisfai deadline extension !\n\nbisfai deadlin...,not_spam,2,0,"[bisfai, deadline, extension, !, bisfai, deadl..."
2,"incredible opportunity\n\ndear friend , recei...",spam,3,1,"[incredible, opportunity, dear, friend, ,, rec..."
3,""" life without debt ""\n\npardon intrusion . o...",spam,4,1,"[``, life, without, debt, ``, pardon, intrusio..."
4,"query :\n\ndear linguist , work double negati...",not_spam,5,0,"[query, :, dear, linguist, ,, work, double, ne..."


In [63]:
test_df.head()

,Text,Label,Ind,label_tag,tokenized_text
0,here\n\nuniversity degree programs increase p...,spam,1,1,"[here, university, degree, programs, increase,..."
1,"endanger language - edinburgh , sept 98 - cal...",not_spam,2,0,"[endanger, language, -, edinburgh, ,, sept, 98..."
2,nels 29 - - call for paper\n\n* * * * * * * *...,not_spam,3,1,"[nels, 29, -, -, call, for, paper, *, *, *, *,..."
3,kornfilt : turkish\n\njaklin kornfilt ( 1997 ...,not_spam,4,1,"[kornfilt, :, turkish, jaklin, kornfilt, (, 19..."
4,9th intl congress of linguist\n\nistituto di ...,not_spam,5,0,"[9th, intl, congress, of, linguist, istituto, ..."


In [64]:
#Remove speacial characters
#Check stop words and special characters
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')
#special=['!','@','#','$','%','^','&','*','(',')',':','<','>',',','.',';','{','}','|','_','-','+','=','`','~',"--","'",'"','[',']','\n','/','\\']
special=[',','.',';','_','-','=','*']
for item  in train_df['tokenized_text']:
    item_new=[]
    for word in item:
        word=str(word)
        if (word in stop_words) or (word in special):
            item.remove(word)
for item  in test_df['tokenized_text']:
    item_new=[]
    for word in item:
        word=str(word)
        if (word in stop_words) or (word in special):
            item.remove(word)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/prashanthimallijula/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [65]:
#Remove speacial characters
#Check stop words and special characters
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')
#special=['!','@','#','$','%','^','&','*','(',')',':','<','>',',','.',';','{','}','|','_','-','+','=','`','~',"--","'",'"','[',']','\n','/','\\']
special=[',','.',';','_','-','=','*']
for item  in train_df['tokenized_text']:
    item_new=[]
    for word in item:
        word=str(word)
        if (word in stop_words) or (word in special):
            item.remove(word)
for item  in test_df['tokenized_text']:
    item_new=[]
    for word in item:
        word=str(word)
        if (word in stop_words) or (word in special):
            item.remove(word)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/prashanthimallijula/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [66]:
train_df.head()

,Text,Label,Ind,label_tag,tokenized_text
0,here\n\nuniversity degree programs increase p...,spam,1,1,"[university, degree, programs, increase, perso..."
1,bisfai deadline extension !\n\nbisfai deadlin...,not_spam,2,0,"[bisfai, deadline, extension, !, bisfai, deadl..."
2,"incredible opportunity\n\ndear friend , recei...",spam,3,1,"[incredible, opportunity, dear, friend, receiv..."
3,""" life without debt ""\n\npardon intrusion . o...",spam,4,1,"[``, life, without, debt, ``, pardon, intrusio..."
4,"query :\n\ndear linguist , work double negati...",not_spam,5,0,"[query, :, dear, linguist, work, double, negat..."


In [67]:
test_df.head()

,Text,Label,Ind,label_tag,tokenized_text
0,here\n\nuniversity degree programs increase p...,spam,1,1,"[university, degree, programs, increase, perso..."
1,"endanger language - edinburgh , sept 98 - cal...",not_spam,2,0,"[endanger, language, edinburgh, sept, 98, call..."
2,nels 29 - - call for paper\n\n* * * * * * * *...,not_spam,3,1,"[nels, 29, call, paper, n, e, l, 29, annual, m..."
3,kornfilt : turkish\n\njaklin kornfilt ( 1997 ...,not_spam,4,1,"[kornfilt, :, turkish, jaklin, kornfilt, (, 19..."
4,9th intl congress of linguist\n\nistituto di ...,not_spam,5,0,"[9th, intl, congress, linguist, istituto, di, ..."


Calculate the required counts of spam and non spam and toal counts

# Feature engineering

For text data procesas Machine sing we need to convert the data into numeric for machine learning modelas run with numeric attributes 

In [68]:
#Getting spam and not spam word count from train data to train the model as a part feature enginerring
for i in range(len(train_df.Label)):
        if(train_df.at[i,'Label'] == 'spam'):
            for word in train_df.Text[i].split(" "):
                spam_counts[word] += 1
                total_counts[word] += 1
        else:
            for word in train_df.Text[i].split(" "):
                not_spam_counts[word] += 1
                total_counts[word] += 1

Below are the most common words in spam messages

In [69]:
print(spam_counts.most_common()[0:40])

[('_', 39231), ('.', 21396), ('*', 19064), ('-', 18913), (',', 12194), ('=', 9069), ('!', 6644), ('/', 4640), ('$', 4448), ("'", 3075), (':', 2834), ('"', 2667), ('+', 2660), ('0', 1958), ('20', 1770), (')', 1685), ('(', 1595), ('order', 1500), ('#', 1470), ('~', 1365), ('report', 1314), ('address', 1299), ('our', 1181), ('mail', 1171), ('email', 1152), ("'s", 1106), ('program', 1045), ('send', 1033), ('>', 992), ('free', 945), ('list', 938), ('receive', 886), ('money', 871), ('name', 868), ('?', 851), (';', 774), ('business', 751), ('3d', 736), ('one', 732), ('1', 731)]


Calculate spam and non spam ratio for eachspam words that are commonly used

In [70]:
for k,v in list(total_counts.most_common()):
    if(v > 100):
        spam_not_ratio = spam_counts[k] / float(not_spam_counts[k]+1)
        spam_not_ratios[k] = spam_not_ratio

Calculate the log value for ratio inorder to get the most used words as features

In [71]:
for word,percent in spam_not_ratios.most_common():
    if(percent > 1):
        spam_not_ratios[word] = np.log(percent)
    else:
        spam_not_ratios[word] = -np.log((1 / (percent+0.01)))

In [72]:
print(list(reversed(spam_not_ratios.most_common()))[0:30])

[('verb', -4.605170185988092), ('phonology', -4.605170185988092), ('syntax', -4.605170185988092), ('translation', -4.605170185988092), ('semantic', -4.605170185988092), ('sentence', -4.605170185988092), ('workshop', -4.605170185988092), ('discourse', -4.605170185988092), ('linguist', -4.605170185988092), ('abstract', -4.605170185988092), ('grammar', -4.605170185988092), ('theory', -4.605170185988092), ('linguistic', -4.605170185988092), ('french', -3.983481969410291), ('linguistics', -3.9670827826111736), ('spanish', -3.9314410912852478), ('august', -3.921875301871158), ('dictionary', -3.916985794770275), ('science', -3.8833378574292876), ('language', -3.8082548198206565), ('japanese', -3.8011223093947044), ('speaker', -3.776477513431922), ('german', -3.7462779928199685), ('structure', -3.6346396851769396), ('english', -3.586982690639934), ('query', -3.526166368708358), ('et', -3.519715981897493), ('papers', -3.5109826833003375), ('ftp', -3.506557897319982), ('1995', -3.464885200919413

Get unique words as features in the dataframe

In [73]:
word_array = set(total_counts.keys())
print(len(word_array ))

27336


In [74]:
word_numpy_array = np.zeros((1, len(word_array)))
print(word_numpy_array.shape)
print(word_numpy_array)

(1, 27336)
[[0. 0. 0. ... 0. 0. 0.]]


In [75]:
#  Maps a word to its column in the word array
word_column_dict = {}
new_word_column_dict={}
for i, word in enumerate(word_array):
    # {key: value} is {word: column}
    word_column_dict[word] = i
c=0
for k,v in word_column_dict.items():
    if v>=1500:
        new_word_column_dict[k]=v
        c=c+1
print(c)        

25836


In [76]:
sorted_IG={}
sorted_new_word_column_dict = sorted(new_word_column_dict.items(), key=lambda kv: kv[1])
c=0
#for k,v in reversed(sorted_new_word_column_dict):
#         print(k,v)
#         c=c+1
#print('length :',c)
c=0
attri_dict={}
for k,v in reversed(sorted_new_word_column_dict):
    if c<=100:
        attri_dict[k]=v
        c=c+1
for k,v in attri_dict.items():
    print(k,v)

negate 27335
evaluated 27334
skovlund 27333
organisation 27332
sgds 27331
antonium 27330
mc4 27329
tokihisa 27328
77-89 27327
accentuation 27326
cath 27325
5dd 27324
verdugo 27323
winslet 27322
e270-1866 27321
odense 27320
fashion 27319
7621 27318
irradicate 27317
586-9313
 27316
75pf 27315
819-823 27314
defeat 27313
linkoep 27312
translational 27311
counter-bleed 27310
distribution 27309
w6 27308
measly 27307
barley 27306
ayyaalum 27305
228 27304
hilbert 27303
duerfen 27302
oishus 27301
lynne 27300
7243 27299
hamburg 27298
mlrlm 27297
6-954 27296
!

message 27295
wilder 27294
481-9891 27293
capital
 27292
inhabitant 27291
left 27290
dartmouth 27289
wilshire 27288
clean-burn 27287
4654 27286
sanford 27285
prochain 27284
wide-scale 27283
lctl 27282
position

linguistic 27281
1848-1905 27280
kurasawa 27279
frederick 27278
meta 27277
multifactorial 27276
commands 27275
affiliation 27274
lexically 27273
vernon 27272
expereince 27271
son 27270
ted 27269
cleopatra 27268
d8 27267
dandruff 272

In [77]:
att_values=[]
for k, v in attri_dict.items():
    att_values=[]
    for item in train_df['tokenized_text']:
        if k in item:
            att_values.append(1)
        else:
            att_values.append(0)
    train_df[k]=att_values

In [78]:
print(train_df.head())

                                                Text     Label  Ind  \
0   here\n\nuniversity degree programs increase p...      spam    1   
1   bisfai deadline extension !\n\nbisfai deadlin...  not_spam    2   
2   incredible opportunity\n\ndear friend , recei...      spam    3   
3   " life without debt "\n\npardon intrusion . o...      spam    4   
4   query :\n\ndear linguist , work double negati...  not_spam    5   

   label_tag                                     tokenized_text  negate  \
0          1  [university, degree, programs, increase, perso...       0   
1          0  [bisfai, deadline, extension, !, bisfai, deadl...       0   
2          1  [incredible, opportunity, dear, friend, receiv...       0   
3          1  [``, life, without, debt, ``, pardon, intrusio...       0   
4          0  [query, :, dear, linguist, work, double, negat...       1   

   evaluated  skovlund  organisation  sgds  ...  tuebingen  karvonen  212-591  \
0          0         0             0     

In [79]:
#train_df.set_index('Ind', inplace=True)

Add the trained features to test data as well. so that we can test on this data

In [80]:
att_values=[]
for k, v in attri_dict.items():
    att_values=[]
    for item in test_df['tokenized_text']:
        if k in item:
            att_values.append(1)
        else:
            att_values.append(0)
    test_df[k]=att_values

# Start on Ml alogrithm-Neural network -FW BW Propagation

In [81]:
#Convert labels into numpy array
l= train_df['label_tag'].tolist()
Y = np.array(l)
Y.shape

(949,)

In [82]:
#Convert input features into numpy array
X =train_df[train_df.columns[5:]]
X=np.array(X)
X.shape
X.shape[0]

949

In [83]:
#import requird Intiallize weights and bias to random values intitlally based on the size of train data we have 
import time
import sys
feature_set = X
labels = Y 
labels = labels.reshape(X.shape[0],1) 
weights = np.random.rand(X.shape[1], 1)  
bias = np.random.rand(1)  
learning_rate = 0.05  
epoch=20000

In [84]:
#Define sifmoid function which is activate function for nn
def sigmoid(x):  
    return 1/(1+np.exp(-x))
def sigmoid_der(x):  
    return sigmoid(x)*(1-sigmoid(x))

In [85]:
#FORWARD AND BACKWARD PROPAGATION METHOD STARTS HERE epoche values is given maunally.
for epoch in range(epoch):  
    inputs = feature_set

    # forward path
    Input_hidden = np.dot(feature_set, weights) + bias

    #forward path 2
    output = sigmoid(Input_hidden)

    # backpropagation find the error rate
    error = output - labels

    print(error.sum())
    # backpropagation  based on the error rate
    dcost_dpred = error
    output_derivative = sigmoid_der(output)

    output_delta = error * output_derivative

    inputs = feature_set.T
#Final weights
    weights -= learning_rate * np.dot(inputs, output_delta)
#Calculate the bias
    for num in output_delta:
        bias -= learning_rate * num
    

66.28589010853932
-105.70590786996169
182.88857374841746
-252.76788687254057
348.54039567214727
-328.18254214535943
388.0784836897783
-313.83849996535344
382.00538951051277
-317.07986337940474
382.62538900288365
-315.6430063927318
381.11131004857845
-315.1133144648708
379.8138002281304
-314.1387781418655
378.2089309574261
-313.0799761596069
376.4885007283066
-311.8900262186785
374.66995444948066
-310.6251341967968
372.81029110737035
-309.3183187132078
370.94964578537247
-308.00372031090524
369.1212903169484
-306.70741829443546
367.34806642402793
-305.44799330413457
365.6438599299923
-304.2367688801577
364.0151598656719
-303.07923132875374
362.4630661420715
-301.9765920040177
360.9851121269594
-300.92723807747024
359.5767466419434
-299.9278951405125
358.23242097231946
-298.97447848743474
356.9463240680983
-298.06266912586335
355.7128391992088
-297.1882738328947
354.5267980542853
-296.3474282336698
353.38359689017454
-295.53669249840425
352.2792243983406
-294.7530776418606
351.2102369861

In [86]:
#slope = input x dcost_dpred x dpred_dz 
output_delta = error * output_derivative  
inputs = feature_set.T  
weights -= learning_rate * np.dot(inputs, output_delta) 

In [87]:
print(len(test_df))

299


In [88]:
#Testing the algorithm on test dataframe we have prediction results displayed for each entry also we have deifned ones with <40% will be non spam and others will be spam
for i in range(len(test_df)):
    list1=test_df.iloc[i]
    list2=list1[5:]
    X =list2
    X=np.array(X,dtype=object)  
    X = X.astype(float)
    result = sigmoid(np.dot(X, weights) + bias) 
    if result >=0.5:
         print(result ,":" "Spam") 
    else:
        print(result ,":" "Not spam") 
    

[0.82570702] :Spam
[0.82570702] :Spam
[0.1035259] :Not spam
[0.74507436] :Spam
[0.82570702] :Spam
[0.31242664] :Not spam
[0.74047577] :Spam
[0.82570702] :Spam
[0.00021081] :Not spam
[0.07551698] :Not spam
[0.82570702] :Spam
[0.82570702] :Spam
[0.82570702] :Spam
[0.82570702] :Spam
[0.82570702] :Spam
[0.82570702] :Spam
[0.0013045] :Not spam
[0.82570702] :Spam
[0.82570702] :Spam
[0.78793289] :Spam
[0.99944785] :Spam
[0.82570702] :Spam
[0.82570702] :Spam
[0.1035259] :Not spam
[0.1035259] :Not spam
[0.82570702] :Spam
[0.82570702] :Spam
[0.1035259] :Not spam
[0.07551698] :Not spam
[0.82570702] :Spam
[0.82570702] :Spam
[0.31242664] :Not spam
[0.96624032] :Spam
[0.82570702] :Spam
[0.82570702] :Spam
[0.00204018] :Not spam
[0.1035259] :Not spam
[0.77017023] :Spam
[0.82570702] :Spam
[0.82570702] :Spam
[0.82570702] :Spam
[0.82570702] :Spam
[0.00027229] :Not spam
[0.82570702] :Spam
[0.1035259] :Not spam
[0.82570702] :Spam
[0.82570702] :Spam
[0.79488318] :Spam
[0.82570702] :Spam
[0.82570702] :Spam
[